# deep identity mapping

learning receptive field parameters from inputs (white-noise videos) and outputs (spike trains) of linear-nonlinear neuron models with parameterized linear filters

## simplistic setup: 
- model parameters $\theta$ are the full spatiotemporal kernel (of size d*d, with d < 10), 
- summary statistic $x_0$ is spike-triggered average
- for sufficiently long simulations, posterior mean is just data summary statistic $x_0$. 
- purely spatial kernel for now (kernel-size in temporal dimentions fixed to 1)  
- *very* basic spiking non-linearity and noise model: threshold crossing, no spiking noise


In [ ]:
%%capture
# notebook currently depends on code found only in feature_maprf-branch of lfi_models !

import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.utils.io as io
import delfi.summarystats as ds
import matplotlib.pyplot as plt
import numpy as np
import lfimodels.maprf.utils as utils

from lfimodels.maprf.maprf import maprf
from lfimodels.maprf.maprfStats import maprfStats
from delfi.utils.viz import plot_pdf

%matplotlib inline

In [ ]:

seed = 42

d = 51 # edge length of (quadratic) receptive field
parametrization = 'gaussian' # RF is Gaussian bump with specific mean and cov 

filter_shape = np.array((d,d,1))
m = maprf(filter_shape=filter_shape, 
          parametrization=parametrization,
          seed=seed, 
          duration=1000 )

if parametrization=='gaussian':
    p = dd.GaussianRF(ab=[-2,-0],
                      dd=[-d//2,d//2,-d//2,d//2],
                      ks = np.array([3,3]), cd = [-.9,0.9])
elif parametrization=='full':
    p = dd.Gaussian(m=np.zeros(d*d+1), P=1/100*np.eye(d*d+1))

s = maprfStats(n_summary=m.n_params)
g = dg.Default(model=m, prior=p, summary=s)

true_params, labels_params = utils.obs_params(filter_shape,parametrization)
true_params = np.array([-d, -d//4,-d//4,10,15,0.7])
obs = m.gen_single(true_params)
obs_stats = s.calc([obs])

h_true = m.params_to_rf(true_params[1:])
plt.figure(figsize=(16,7))
plt.subplot(1,3,1)
plt.plot(np.dot( obs['I'], h_true) + true_params[0])
plt.title('neural activation function over observed data')
plt.subplot(1,3,2)
plt.imshow(h_true.reshape(d,d), interpolation='None')
plt.title('ground-truth filter')
plt.subplot(1,3,3)
plt.imshow(obs_stats.T[1:].reshape(filter_shape[0],filter_shape[1]), 
          interpolation='None')
plt.title('summary statistics')
plt.show()

# bunch of example prior draws
plt.figure(figsize=(16,7))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(m.params_to_rf(p.gen()[0,1:]).reshape(d,d), interpolation='None')
plt.subplot(2,5,3)
plt.title('RF prior samples')
plt.show()


obs['data'].mean()

In [ ]:

res = infer.SNPE(g, obs=obs_stats, n_hiddens=[20,20])

out = res.run(10000, n_rounds=1, minibatch=50, epochs=1000)
posterior = res.predict(obs_stats)

plt.figure(figsize=(16,6))
plt.subplot(1,4,1)
h_prior = m.params_to_rf(p.mean[1:])
plt.imshow(h_prior.reshape(d,d), interpolation='None')
plt.title('prior mean filter')
plt.subplot(1,4,2)
h_est =  m.params_to_rf(posterior.calc_mean_and_cov()[0][1:])
plt.imshow(h_est.reshape(d,d), interpolation='None')
plt.title('posterior mean filter')
plt.subplot(1,4,3)
plt.imshow(h_true.reshape(d,d), 
          interpolation='None')
plt.title('true spatial filter')
plt.subplot(1,4,4)
plt.imshow(obs_stats.T[1:].reshape(filter_shape[0],filter_shape[1]), 
          interpolation='None')
plt.title('x0')
plt.show()

#print('prior mean\n', p.mean[1:].reshape(-1,1))
#print('post mean\n', posterior.calc_mean_and_cov()[0][1:].reshape(-1,1))
#print('true pars\n', true_params[1:].reshape(-1,1))
#print('x0\n', obs_stats.T[1:]) 

posterior = res.predict(obs_stats)
plot_pdf(posterior.xs[0], lims=[-5,5], gt=true_params, figsize=(12,12));
tmp=posterior.xs[0]
tmp.eval(true_params.reshape(1,-1))

# construction site

In [ ]:
import numpy as np
import lasagne
import lasagne.init as linit
import lasagne.layers as ll
import lasagne.nonlinearities as lnl
import theano
import theano.tensor as tt
import collections
import delfi.distribution as dd
import delfi.neuralnet.layers as dl
from delfi.neuralnet.Trainer import Trainer

from delfi.utils.odict import first, last, nth

dtype = theano.config.floatX

class cNeuralNet(object):
    def __init__(self, n_inputs, n_outputs, n_components=1, n_filters=[10,10], n_hiddens=[10, 10],
                 n_rnn=None, seed=None, svi=True):
        """Initialize a mixture density network with custom layers

        Parameters
        ----------
        n_inputs : (int, int)
            Dimensionality of input
        n_outputs : int
            Dimensionality of output
        n_components : int
            Number of components of the mixture density
        n_filters : list of ints
            Number of filters  per convolutional layer
        n_hiddens : list of ints
            Number of hidden units per fully connected layer
        n_rnn : None or int
            Number of RNN units
        seed : int or None
            If provided, random number generator will be seeded
        svi : bool
            Whether to use SVI version or not
        """
        self.n_components = n_components
        self.n_hiddens = n_hiddens
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.n_rnn = n_rnn
        self.svi = svi

        self.seed = seed
        if seed is not None:
            self.rng = np.random.RandomState(seed=seed)
        else:
            self.rng = np.random.RandomState()
        lasagne.random.set_rng(self.rng)

        # placeholders
        # stats : input placeholder, (batch, self.n_inputs)
        # params : output placeholder, (batch, self.n_outputs)
        self.stats = tt.tensor4('stats', dtype=dtype)
        self.params = tt.matrix('params', dtype=dtype)

        # compose layers
        self.layer = collections.OrderedDict()

        # input layer
        self.layer['input'] = ll.InputLayer(
            (None, 1, self.n_inputs[0], self.n_inputs[1]), input_var=self.stats)
        # ... or substitute NaN for zero
        # ... or learn replacement values
        # ... or a recurrent neural net

        # convolutional layers
        for l in range(len(n_filters)):
            self.layer['conv_' + str(l + 1)] = ll.Conv2DLayer(
                           incoming=last(self.layer), 
                           num_filters=n_filters[l], 
                           filter_size=3, 
                           stride=(2, 2), 
                           pad=0, 
                           untie_biases=False, 
                           W=lasagne.init.GlorotUniform(), 
                           b=lasagne.init.Constant(0.), 
                           nonlinearity=lasagne.nonlinearities.rectify, 
                           flip_filters=True, 
                           convolution=theano.tensor.nnet.conv2d)
        
        self.layer['flatten'] = ll.FlattenLayer(
                            incoming=last(self.layer), 
                            outdim = 2)
        # if len(n_filters)==0, this directly reshapes the input layer to 2D
        
        # hidden layers
        for l in range(len(n_hiddens)):
            self.layer['hidden_' + str(l + 1)] = dl.FullyConnectedLayer(
                last(self.layer), n_units=n_hiddens[l],
                svi=svi, name='h' + str(l + 1))
            
            
        last_hidden = last(self.layer)

        # mixture layers
        self.layer['mixture_weights'] = dl.MixtureWeightsLayer(
            last_hidden, n_units=n_components, actfun=lnl.softmax, svi=svi,
            name='weights')
        self.layer['mixture_means'] = dl.MixtureMeansLayer(
            last_hidden, n_components=n_components, n_dim=n_outputs, svi=svi,
            name='means')
        self.layer['mixture_precisions'] = dl.MixturePrecisionsLayer(
            last_hidden, n_components=n_components, n_dim=n_outputs, svi=svi,
            name='precisions')
        last_mog = [self.layer['mixture_weights'],
                    self.layer['mixture_means'],
                    self.layer['mixture_precisions']]

        # mixture parameters
        # a : weights, matrix with shape (batch, n_components)
        # ms : means, list of len n_components with (batch, n_dim, n_dim)
        # Us : precision factors, n_components list with (batch, n_dim, n_dim)
        # ldetUs : log determinants of precisions, n_comp list with (batch, )
        self.a, self.ms, precision_out = ll.get_output(last_mog,
                                                       deterministic=False)

        self.Us = precision_out['Us']
        self.ldetUs = precision_out['ldetUs']

        self.comps = {
            **{'a': self.a},
            **{'m' + str(i): self.ms[i] for i in range(self.n_components)},
            **{'U' + str(i): self.Us[i] for i in range(self.n_components)}}

        # log probability of y given the mixture distribution
        # lprobs_comps : log probs per component, list of len n_components with (batch, )
        # probs : log probs of mixture, (batch, )
        
        self.lprobs_comps = [-0.5 * tt.sum(tt.sum((self.params - m).dimshuffle(
            [0, 'x', 1]) * U, axis=2)**2, axis=1) + ldetU
            for m, U, ldetU in zip(self.ms, self.Us, self.ldetUs)]
        self.lprobs = tt.log(tt.sum(tt.exp(tt.stack(self.lprobs_comps,
                                                    axis=1) + tt.log(self.a)),
                                    axis=1)) - (0.5 * self.n_outputs * np.log(2 * np.pi))

        # the quantities from above again, but with deterministic=True
        # --- in the svi case, this will disable injection of randomness;
        # the mean of weights is used instead
        self.da, self.dms, dprecision_out = ll.get_output(last_mog,
                                                          deterministic=True)
        self.dUs = dprecision_out['Us']
        self.dldetUs = dprecision_out['ldetUs']
        self.dcomps = {
            **{'a': self.da},
            **{'m' + str(i): self.dms[i] for i in range(self.n_components)},
            **{'U' + str(i): self.dUs[i] for i in range(self.n_components)}}
        self.dlprobs_comps = [-0.5 * tt.sum(tt.sum((self.params - m).dimshuffle(
            [0, 'x', 1]) * U, axis=2)**2, axis=1) + ldetU
            for m, U, ldetU in zip(self.dms, self.dUs, self.dldetUs)]
        self.dlprobs = tt.log(tt.sum(tt.exp(tt.stack(self.dlprobs_comps,
                                                     axis=1) + tt.log(self.da)),
                                     axis=1)) - (0.5 * self.n_outputs * np.log(2 * np.pi))        

        # parameters of network
        self.aps = ll.get_all_params(last_mog)  # all parameters
        self.mps = ll.get_all_params(last_mog, mp=True)  # means
        self.sps = ll.get_all_params(last_mog, sp=True)  # log stds

        # weight and bias parameter sets as seperate lists
        self.mps_wp = ll.get_all_params(last_mog, mp=True, wp=True)
        self.sps_wp = ll.get_all_params(last_mog, sp=True, wp=True)
        self.mps_bp = ll.get_all_params(last_mog, mp=True, bp=True)
        self.sps_bp = ll.get_all_params(last_mog, sp=True, bp=True)

        # theano functions
        self.compile_funs()

    def compile_funs(self):
        """Compiles theano functions"""
        self._f_eval_comps = theano.function(
            inputs=[self.stats],
            outputs=self.dcomps)
        self._f_eval_lprobs = theano.function(
            inputs=[self.params, self.stats],
            outputs=self.dlprobs)

    def eval_comps(self, stats):
        """Evaluate the parameters of all mixture components at given inputs

        Parameters
        ----------
        stats : np.array
            rows are input locations

        Returns
        -------
        mixing coefficients, means and scale matrices
        """
        return self._f_eval_comps(stats.astype(dtype))

    def eval_lprobs(self, params, stats):
        """Evaluate log probabilities for given input-output pairs.

        Parameters
        ----------
        params : np.array
        stats : np.array

        Returns
        -------
        log probabilities : log p(params|stats)
        """
        return self._f_eval_lprobs(params.astype(dtype), stats.astype(dtype))

    def get_mog(self, stats, n_samples=None):
        """Return the conditional MoG at location x

        Parameters
        ----------
        stats : np.array
            single input location
        n_samples : None or int
            ...
        """
        assert stats.shape[0] == 1, 'x.shape[0] needs to be 1'

        comps = self.eval_comps(stats)
        a = comps['a'][0]
        ms = [comps['m' + str(i)][0] for i in range(self.n_components)]
        Us = [comps['U' + str(i)][0] for i in range(self.n_components)]

        return dd.MoG(a=a, ms=ms, Us=Us, seed=self.gen_newseed())

    def gen_newseed(self):
        """Generates a new random seed"""
        if self.seed is None:
            return None
        else:
            return self.rng.randint(0, 2**31)

    @property
    def params_dict(self):
        """Getter for params as dict"""
        pdict = {}
        for p in self.aps:
            pdict[str(p)] = p.get_value()
        return pdict

    @params_dict.setter
    def params_dict(self, pdict):
        """Setter for params as dict"""
        for p in self.aps:
            if str(p) in pdict.keys():
                p.set_value(pdict[str(p)])

    @property
    def spec_dict(self):
        """Specs as dict"""
        return {'n_inputs': self.n_inputs,
                'n_outputs': self.n_outputs,
                'n_components': self.n_components,
                'n_hiddens': self.n_hiddens,
                'n_rnn': self.n_rnn,
                'seed': self.seed,
                'svi': self.svi}
    


In [ ]:
n_inputs = [d,d]
n_outputs = m.n_params
n_components=1
n_hiddens=[10,10]
n_filters=[10,10,10]
n_train_round = 1

trn_data = g.gen(1000)
trn_data = (trn_data[0], trn_data[1][:,1:].reshape(-1,1,d,d))

network = cNeuralNet(n_inputs, 
                     n_outputs, 
                     n_components, 
                     n_filters=n_filters, 
                     n_hiddens=n_hiddens,
                     seed=seed,
                     svi=False)
loss = -tt.mean(network.lprobs)


trn_inputs = [network.params, network.stats]
t = Trainer(network, loss,
            trn_data=trn_data, trn_inputs=trn_inputs)

epochs = 200
minibatch = 50
logs=[]
logs.append(t.train(epochs=epochs, minibatch=minibatch))

posterior = network.get_mog(obs_stats[0,1:].reshape(1,1,d,d))

plt.figure(figsize=(16,6))
plt.subplot(1,4,1)
h_prior = m.params_to_rf(p.mean[1:])
plt.imshow(h_prior.reshape(d,d), interpolation='None')
plt.title('prior mean filter')
plt.subplot(1,4,2)
h_est =  m.params_to_rf(posterior.calc_mean_and_cov()[0][1:])
plt.imshow(h_est.reshape(d,d), interpolation='None')
plt.title('posterior mean filter')
plt.subplot(1,4,3)
plt.imshow(h_true.reshape(d,d), 
          interpolation='None')
plt.title('true spatial filter')
plt.subplot(1,4,4)
plt.imshow(obs_stats.T[1:].reshape(filter_shape[0],filter_shape[1]), 
          interpolation='None')
plt.title('x0')
plt.show()

plot_pdf(posterior.xs[0], lims=[-5,5], gt=true_params, figsize=(12,12));


# bunch of example posterior draws
plt.figure(figsize=(16,7))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(m.params_to_rf(posterior.xs[0].gen()[0,1:]).reshape(d,d), interpolation='None')
plt.subplot(2,5,3)
plt.title('RF posterior samples')
plt.show()


In [ ]:
import numpy as np
import scipy.signal as ss
from scipy.stats import multivariate_normal as mvn

from delfi.simulator.BaseSimulator import BaseSimulator

import maprf.rfs.v1 as v1
import maprf.glm as glm
from maprf.generation import Generator

class maprf(BaseSimulator):
    
    def __init__(self, 
        duration=10, 
        filter_shape=np.array((1,1,1)), 
        parametrization='full',
        seed=None):
        """GLM simulator

        Parameters
        ----------
        duration : int
            Duration of traces in ms
        size_filter : (Nx, Ny, T)
            Size of spatiotemporal filter
        parametrization : string
            parametrization of linear filter
        seed : int or None
            If set, randomness across runs is disabled
        """
        super(maprf, self).__init__(dim_param=np.prod(filter_shape).astype(np.int)+1, 
                                    seed=seed)

        self.duration = duration
        self.filter_shape = filter_shape
        self.len_filter  = filter_shape[-1]
        self.size_filter = np.prod(filter_shape[:-1]).astype(np.int)
        self.parametrization = parametrization
        
        # parameters that globally govern the simulations
        self.dt = 0.025
        self.t = np.arange(0, self.duration, self.dt)                
        
        # input: gaussian white noise N(0, 1)
        self.rng_input = np.random.RandomState(seed=self.gen_newseed())        
        self.I = self.rng_input.randn(len(self.t),self.size_filter)        
                
        if parametrization=='full':

            self.n_params = np.prod(filter_shape).astype(np.int) + 1

        elif  parametrization=='gaussian':

            self.n_params = self.len_filter + 5 # offset + 2x (2D mean) + 3x (cov)

        elif  parametrization=='gabor':
            
            self.n_params = 10 # 'glm':         'bias', 'binsize'
                              # 'kernel': 's': 'angle','freq','gain','phase','ratio','width',
                              #           't': 'tau'
                    
            kernel = v1.SimpleLinear_full_kt()
            spikes = glm.Poisson()

            self._gen = Generator(kernel, spikes, seed=seed)
            d = self.filter_shape[0]
            self._gen.axis_x = np.linspace(-5, 5, d)
            d = self.filter_shape[1]
            self._gen.axis_y = np.linspace(-5, 5, d)
            self._gen.axis_t = self.dt * np.arange(0,self.len_filter)
            
            self._gen.grid_x, self._gen.grid_y = np.meshgrid(
                                 self._gen.axis_x, 
                                 self._gen.axis_y)            
            
            self._gen.build()
            
            self._gen.x = self.I.reshape(len(self.t), 
                                         self.filter_shape[0], 
                                         self.filter_shape[1])
            
                
        else:
            raise NotImplementedError        

    def gen_single(self, params):
        """Forward model for simulator for single parameter set

        Parameters
        ----------
        params : list or np.array, 1d of length dim_param
            Parameter vector

        Returns
        -------
        dict : dictionary with data
            The dictionary must contain a key data that contains the results of the forward run. Additional entries can be present.
        """
        params = np.asarray(params)

        assert params.ndim == 1, 'params.ndim must be 1'
        assert params.shape[0] == self.n_params, 'params.shape[0] must be dim params long'

        if self.parametrization=='full':
            
            b0 = params[0]
            b0.astype(float)            
            h = params[1:]
            h.astype(float)

        # construct h from parameters
        elif  self.parametrization=='gaussian':

            b0 = params[0]
            b0.astype(float)
            h = self.params_to_rf(params[1:])

        if self.parametrization in ('full', 'gaussian'):    
        
            assert self.filter_shape[2] == 1 # else not yet implemented            
        
            # simulation
            assert self.len_filter == 1 # temporal filtering not implemented yet!
            psi = b0 + np.dot(self.I, h) #ss.lfilter(h, 1, self.I, axis=0).sum(axis=1)

            # psi goes through a sigmoid non-linearity: firing probability
            z = 1 /(1 + np.exp(-psi)).reshape(-1,1)        

            # sample the spikes
            N = 1   # number of trials
            y = self.rng.uniform(size=(len(self.t), N)) < z
            y = np.sum(y, axis=1)
            
        elif self.parametrization == 'gabor':
                        
            params_dict = {'glm': {'bias': params[0], 
                                   'binsize': params[1]},
                          'kernel': {'s': {'angle': params[2],
                                           'freq': params[3],
                                           'gain': params[4],
                                           'phase': params[5],
                                           'ratio': params[6],
                                           'width': params[7]},
                                     't': {'value': params[8:].reshape(-1)}}}   
            
            self._gen.set_params(params_dict)
            self._gen._sample()
            y = self._gen.y.copy()
            
        return {'data': y, 'I': self.I}

    def params_to_rf(self, params_rf):

        if self.parametrization=='full':
            
            h = params_rf

        elif self.parametrization=='gaussian':

            c = params_rf[4] * params_rf[2] * params_rf[3] # corr to cov
            C = np.array([[params_rf[2]**2,c],[c,params_rf[3]**2]])
            dx, dy = self.filter_shape[0]//2, self.filter_shape[1]//2
            assert self.filter_shape[0]>2*dx and self.filter_shape[1]>2*dy

            dx, dy = range(-dx,dx+1), range(-dy,dy+1)
            h = mvn.pdf(np.mgrid[dx, dy].reshape(2,-1).T, 
                        mean=params_rf[:2],  
                        cov=C + 1e-5*np.eye(2))
            h -= h.mean()
            std = h.std() 
            if std>0: 
                h /= std            


            """ # Old Gaussian parametrization with sq. of full cov matrix 
            B = params_rf[2:].reshape(2,2)
            dx, dy = self.filter_shape[0]//2, self.filter_shape[1]//2
            assert self.filter_shape[0]>2*dx and self.filter_shape[1]>2*dy

            dx, dy = range(-dx,dx+1), range(-dy,dy+1)
            h = mvn.pdf(np.mgrid[dx, dy].reshape(2,-1).T, 
                        mean=params_rf[:2],  
                        cov=B.dot(B.T) + 1e-5*np.eye(2))
            h -= h.mean()
            std = h.std() 
            if std>0: 
                h /= std            
            """
        return h        


In [ ]:
from lfimodels.maprf.maprfStats import maprfStats
import theano

seed = 42

s = maprfStats(n_summary=10)

d = 21 # edge length of (quadratic) receptive field
parametrization = 'gabor' # RF is Gaussian bump with specific mean and cov 

filter_shape = np.array((d,d,2))
m = maprf(filter_shape=filter_shape, 
          parametrization=parametrization,
          seed=seed, 
          duration=1000 )

eval_ks = theano.function(list(m._gen.inputs.values()), m._gen.rf.ks, on_unused_input='ignore')
eval_kt = theano.function(list(m._gen.inputs.values()), m._gen.rf.kt, on_unused_input='ignore')
def rf(params): 
    
    ks = eval_ks(bias=params[0], 
                 binsize=params[1],
                 angle=params[2],
                 freq=params[3],
                 gain=params[4],
                 phase=params[5],
                 ratio=params[6],
                 width=params[7])
    kt = eval_kt(value=params[8:])
    
    return ks,kt


pars_true = np.array([-0.5,.025,.7,.3,2.,1.,1.,2.5,1.,0.])

out = m.gen_single(pars_true)
sta = s.calc([out])[0,1:].reshape(d,d)

# 'glm':         'bias', 'binsize'
# 'kernel': 's': 'angle','freq','gain','phase','ratio','width',
#           't': 'value'

p = dd.Uniform(lower = np.array([-1,  0.2, -1, -1, 0, -1, 0, .5, 0, 0]), 
               upper = np.array([ 1,  0.3,  1,  1, 5,  1, 2,  2, 1, 1]))
#p = dd.Gaussian(m=np.zeros(10), P=1/10*np.eye(10))

g = dg.Default(model=m, prior=p, summary=s)

obs = m.gen_single(pars_true)
obs_stats = s.calc([obs])




n_inputs = [d,d]
n_outputs = m.n_params
n_components=1
n_hiddens=[20,20]
n_filters=[16,16,32]
n_train_round = 1

trn_data = g.gen(10000)
trn_data = (trn_data[0], trn_data[1][:,1:].reshape(-1,1,d,d))

network = cNeuralNet(n_inputs, 
                     n_outputs, 
                     n_components, 
                     n_filters=n_filters, 
                     n_hiddens=n_hiddens,
                     seed=seed,
                     svi=False)
loss = -tt.mean(network.lprobs)


trn_inputs = [network.params, network.stats]
t = Trainer(network, loss,
            trn_data=trn_data, trn_inputs=trn_inputs)

epochs = 100
minibatch = 50
logs=[]
logs.append(t.train(epochs=epochs, minibatch=minibatch))

posterior = network.get_mog(obs_stats[0,1:].reshape(1,1,d,d))


# bunch of example prior draws
plt.figure(figsize=(16,10))
for i in range(15):
    plt.subplot(3,5,i+1)
    pars = p.gen().reshape(-1)
    plt.imshow(rf(pars)[0], interpolation='None')
plt.title('RF prior STAs')
plt.show()

plt.figure(figsize=(16,6))
plt.subplot(1,3,1)
plt.imshow(sta, interpolation='None')
plt.title('data STA')
plt.subplot(1,3,2)
plt.imshow(rf(pars_true)[0], interpolation='None')
plt.title('ground-truth posterior RF')
plt.subplot(1,3,3)
plt.imshow(rf(posterior.xs[0].m)[0], interpolation='None')
plt.title('posterior mean STA')
plt.show()

# bunch of example posterior draws
plt.figure(figsize=(16,10))
for i in range(15):
    plt.subplot(3,5,i+1)
    pars = posterior.gen().reshape(-1)
    plt.imshow(rf(pars)[0], interpolation='None')
plt.title('RF posterior STAs')
plt.show()


plot_pdf(posterior, lims=[-5,5], gt=np.asarray(pars_true).reshape(-1), figsize=(12,12));



In [ ]:
from lfimodels.maprf.maprfStats import maprfStats
import theano

seed = 42

s = maprfStats(n_summary=10)

d = 21 # edge length of (quadratic) receptive field
parametrization = 'gabor' # RF is Gaussian bump with specific mean and cov 

filter_shape = np.array((d,d,2))
m = maprf(filter_shape=filter_shape, 
          parametrization=parametrization,
          seed=seed, 
          duration=10000 )

eval_ks = theano.function(list(m._gen.inputs.values()), m._gen.rf.ks, on_unused_input='ignore')
eval_kt = theano.function(list(m._gen.inputs.values()), m._gen.rf.kt, on_unused_input='ignore')
def rf(params): 
    
    ks = eval_ks(bias=params[0], 
                 binsize=params[1],
                 angle=params[2],
                 freq=params[3],
                 gain=params[4],
                 phase=params[5],
                 ratio=params[6],
                 width=params[7])
    kt = eval_kt(value=params[8:])
    
    return ks,kt


pars_true = np.array([-0.5,.025,.7,.3,2.,1.,1.,2.5,1.,0.])

out = m.gen_single(pars_true)
sta = s.calc([out])[0,1:].reshape(d,d)

# 'glm':         'bias', 'binsize'
# 'kernel': 's': 'angle','freq','gain','phase','ratio','width',
#           't': 'value'

p = dd.Uniform(lower = np.array([-1,  0.2, -1, -1, 0, -1, 0, .5, 0, 0]), 
               upper = np.array([ 1,  0.3,  1,  1, 5,  1, 2,  2, 1, 1]))
#p = dd.Gaussian(m=np.zeros(10), P=1/10*np.eye(10))

g = dg.Default(model=m, prior=p, summary=s)

obs = m.gen_single(pars_true)
obs_stats = s.calc([obs])




n_inputs = [d,d]
n_outputs = m.n_params
n_components=1
n_hiddens=[20,20]
n_filters=[16,16,32]
n_train_round = 1

trn_data = g.gen(10000)
trn_data = (trn_data[0], trn_data[1][:,1:].reshape(-1,1,d,d))

network = cNeuralNet(n_inputs, 
                     n_outputs, 
                     n_components, 
                     n_filters=n_filters, 
                     n_hiddens=n_hiddens,
                     seed=seed,
                     svi=False)
loss = -tt.mean(network.lprobs)


trn_inputs = [network.params, network.stats]
t = Trainer(network, loss,
            trn_data=trn_data, trn_inputs=trn_inputs)

epochs = 100
minibatch = 50
logs=[]
logs.append(t.train(epochs=epochs, minibatch=minibatch))

posterior = network.get_mog(obs_stats[0,1:].reshape(1,1,d,d))


# bunch of example prior draws
plt.figure(figsize=(16,10))
for i in range(15):
    plt.subplot(3,5,i+1)
    pars = p.gen().reshape(-1)
    plt.imshow(rf(pars)[0], interpolation='None')
plt.title('RF prior STAs')
plt.show()

plt.figure(figsize=(16,6))
plt.subplot(1,3,1)
plt.imshow(sta, interpolation='None')
plt.title('data STA')
plt.subplot(1,3,2)
plt.imshow(rf(pars_true)[0], interpolation='None')
plt.title('ground-truth posterior RF')
plt.subplot(1,3,3)
plt.imshow(rf(posterior.xs[0].m)[0], interpolation='None')
plt.title('posterior mean STA')
plt.show()

# bunch of example posterior draws
plt.figure(figsize=(16,10))
for i in range(15):
    plt.subplot(3,5,i+1)
    pars = posterior.gen().reshape(-1)
    plt.imshow(rf(pars)[0], interpolation='None')
plt.title('RF posterior STAs')
plt.show()


plot_pdf(posterior, lims=[-5,5], gt=np.asarray(pars_true).reshape(-1), figsize=(12,12));



In [ ]:
from maprf.inference import *

rf = v1.SimpleLinear()
emt = glm.Poisson()


# inference model
inference = Inference(rf, emt)
inference.priors = cfg['priors']
inference.priors['kernel']['t'] = {'mu': np.zeros_like(ax_t), 'sigma': linalg.cholesky(Σ)}


inference.add_sampler(GaborSampler())
inference.add_sampler(KernelSampler())

inference.build(data)
inference.updates
inference.compile()

fname = 'example_trace_1'
with open(fname + '.npy', 'rb') as file:
    __s, __y, __kt = pickle.load(file)
inference.loglik['xo'] = 0
inference.loglik['yo'] = 0
# inference.loglik['dt'] = 0.025
# inference.loglik['bias'] = -1.0
inference.loglik['kt'] = __kt
inference.loglik['vec_A'] = np.zeros(2)  # np.array([2.0, 0.0])
inference.loglik['vec_f'] = np.zeros(2)  # 0.3 * np.array([np.cos(0.7), np.sin(0.7)])
inference.loglik['log_γ'] = 0.0
inference.loglik['log_b'] = np.log(2.5)

frames.set_value(__s)
spikes.set_value(__y)
plt.plot(spikes.get_value())

print(np.sum(__y))